In [1]:
import pysam
import collections
from math import floor, ceil
from pprint import pprint
import sys
import functions 

In [2]:
#lectura del archivo bam 
bam_file = pysam.AlignmentFile('SRR769610_trim.bam','rb')
#header - contiene informacion sobre el alineamiento y los cromosomas
header = bam_file.header.to_dict()
total_reads = bam_file.mapped

#obtencion de reads e informacion por cromosoma
chromosomes = {}
genome_size = 0

for i in header['SQ']:
    chromosome_name = i['SN']
    chromosome_lenght = i['LN']
    genome_size += chromosome_lenght

    #chromosomes[i['SN']] =  #para sacar los reads de un cromosoma se usa fecth
    #i['LN'] es el largo
    reads = bam_file.fetch(i['SN'])
    
     #lista de posiciones de los reads
    pos = []
    for read in reads:
        p = (read.to_dict())['ref_pos']
        pos.append(p)

    #obtencion de bins 
    bins = [functions.pos_to_bin(p) for p in pos]
    o_bins = [functions.pos_to_bin(p, overlap=True) for p in pos]
    c = dict(collections.Counter(bins + o_bins))

    chromosomes[chromosome_name] = dict(sorted(c.items()))


bam_file.close()


In [3]:

for k,v in chromosomes.items():

    dict_aux = {}
    bins = list(v.keys())
    counts = list(v.values())
    
    for i in range(len(v)):
        dict_aux[bins[i]] = functions.bin_to_RPM(counts[i], total_reads) 
        #print(bins[i], counts[i])
    
    chromosomes[k] = dict_aux

In [4]:
chromosomes_bin = {}
for chr_name, values in chromosomes.items():
  #print(chr_name, values)
  bins = functions.bin_group(values, treshold= 2)
  if len(bins) != 0 :
    chromosomes_bin[chr_name] = functions.clustering_bins(bins, treshold = 2)
 #   test_clust = functions.clustering_bins(test, treshold = 2)

In [5]:
#number of clusters
for i in list(chromosomes_bin.values()):
    print(len(i))

2626
164
1078
1544
1898
1664
1306
1173
315
385
18
9
7
8
1
1


In [16]:
x = round(5.76543)
print(x)

6


In [26]:
for i in chromosomes_bin.values():
    for n in range(len(i)):
        
        l = (i[n][-1] - i[n][0])
        
        if ((l+1)%2) == 0:
            print("start:{}, stop: {}, size: {}, lenght: {}".format(i[n][0], i[n][-1], (l+1), (l+2)*25 ))

        else:

            print("start:{}, stop: {}, size: {}, lenght: {}".format(i[n][0], i[n][-1], (l+1), (ceil((l+1)/2))*50))


start:8, stop: 8, size: 1, lenght: 50
start:40, stop: 41, size: 2, lenght: 75
start:49, stop: 59, size: 11, lenght: 300
start:63, stop: 74, size: 12, lenght: 325
start:83, stop: 100, size: 18, lenght: 475
start:103, stop: 114, size: 12, lenght: 325
start:117, stop: 119, size: 3, lenght: 100
start:124, stop: 125, size: 2, lenght: 75
start:128, stop: 129, size: 2, lenght: 75
start:134, stop: 136, size: 3, lenght: 100
start:140, stop: 141, size: 2, lenght: 75
start:145, stop: 156, size: 12, lenght: 325
start:160, stop: 161, size: 2, lenght: 75
start:165, stop: 166, size: 2, lenght: 75
start:169, stop: 176, size: 8, lenght: 225
start:179, stop: 180, size: 2, lenght: 75
start:187, stop: 187, size: 1, lenght: 50
start:190, stop: 199, size: 10, lenght: 275
start:202, stop: 204, size: 3, lenght: 100
start:207, stop: 209, size: 3, lenght: 100
start:217, stop: 218, size: 2, lenght: 75
start:221, stop: 222, size: 2, lenght: 75
start:230, stop: 241, size: 12, lenght: 325
start:246, stop: 260, size

In [8]:
import pandas as pd

In [14]:
pd.DataFrame.from_dict(chromosomes_bin, orient='index')

,0,1,2,3,4,5,6,7,8,9,...,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986
I,[8],[40],"[50, 52, 54, 56, 58]","[64, 66, 68, 70, 72, 74]","[84, 86, 88, 90, 92, 94, 96, 98, 100]","[104, 106, 108, 110, 112, 114]",[118],[124],[128],"[134, 136]",...,"[391849, 391851]",[391859],"[391863, 391865, 391867, 391869, 391871]","[391879, 391881, 391883, 391885]",[391891],"[391895, 391897]",[391901],"[391907, 391909, 391911]","[391917, 391919, 391921, 391923, 391925]",[391929]
Mt,"[0, 2, 4, 6, 8, 10, 12]",[16],"[20, 22]","[26, 28, 30, 32]",[40],"[48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70]","[78, 80, 82, 84, 86, 88, 90]","[94, 96, 98, 100, 102, 104, 106, 108]","[116, 118]","[122, 124]",...,None,None,None,None,None,None,None,None,None,None
II,"[18, 20]",[28],[38],[44],"[48, 50, 52, 54, 56, 58]",[62],[70],"[74, 76]","[80, 84]",[90],...,None,None,None,None,None,None,None,None,None,None
III,[8],"[12, 14]","[28, 30]",[44],[54],"[58, 60, 62]",[70],[78],[84],[88],...,None,None,None,None,None,None,None,None,None,None
IV,[22],[26],"[32, 34, 36, 38, 40, 42, 44]",[48],"[52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 7...",[86],[92],"[98, 100]",[104],"[110, 112, 114]",...,None,None,None,None,None,None,None,None,None,None
V,[22],[208],"[248, 250, 252]","[270, 272, 274, 276, 278, 280, 282, 284, 286, ...",[1320],[1330],[2370],[2392],[2398],[2402],...,None,None,None,None,None,None,None,None,None,None
VI,[6],[32],"[44, 46]",[50],"[54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 7...","[90, 92, 94]","[98, 100, 102, 104, 106, 108, 110, 112]",[116],"[120, 122, 124, 126]","[130, 132, 134]",...,None,None,None,None,None,None,None,None,None,None
VII,[6],"[12, 14]","[22, 24]","[34, 36, 38, 40, 42, 44, 46, 48]",[56],"[60, 62]","[66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 8...","[96, 98]","[104, 106, 108, 110]",[118],...,None,None,None,None,None,None,None,None,None,None
supercont12.8,[2],[14],"[20, 22, 24]","[38, 40, 42]","[50, 52]",[68],"[88, 90]",[102],[106],"[110, 112]",...,None,None,None,None,None,None,None,None,None,None
supercont12.9,"[8, 10, 12, 14]","[18, 20]","[28, 30]",[36],[40],[46],"[52, 54, 56, 58, 60, 62, 64, 66]","[70, 72]","[76, 78]",[84],...,None,None,None,None,None,None,None,None,None,None


In [26]:
len(pos[10]['seq'])

24

In [19]:
pos[0]

{'name': 'SRR769610.25439498',
 'flag': '16',
 'ref_name': 'I',
 'ref_pos': '195',
 'map_quality': '255',
 'cigar': '19M',
 'next_ref_name': '*',
 'next_ref_pos': '0',
 'length': '0',
 'seq': 'TAACCCTAACCCTGCTCTA',
 'qual': '*',
 'tags': ['XA:i:0',
  'MD:Z:19',
  'NM:i:0',
  'XM:i:1',
  'RG:Z:SRR769610_trim',
  'XX:i:1',
  'XY:Z:U',
  'XZ:f:1']}